In [9]:
#Feature Engineering
#creo nuevas variables dentro del mismo mes
#Condimentar a gusto con nuevas variables

#limpio la memoria
rm( list=ls() )
gc()

require("data.table")

#Establezco el Working Directory
setwd("C:/Users/bbvis/Documents/RG/Master/laboratorio")


EnriquecerDataset <- function( dataset , arch_destino )
{
  columnas_originales <-  copy(colnames( dataset ))

  #INICIO de la seccion donde se deben hacer cambios con variables nuevas
  #se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
  #varias formas de combinar Visa_status y Master_status
  dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  #dataset[ , mv_status02       := Master_status +  Visa_status ]
  #dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  #dataset[ , mv_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  #dataset[ , mv_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]

  #dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
  #                                        ifelse( is.na(Master_status), 10, Master_status), 
  #                                        Visa_status)  ]

  #dataset[ , mv_status07       := ifelse( is.na(Master_status), 
  #                                        ifelse( is.na(Visa_status), 10, Visa_status), 
  #                                        Master_status)  ]


    
  #combino MasterCard y Visa
  dataset[ , mv_mfinanciacion_limite := rowSums( cbind( Master_mfinanciacion_limite,  Visa_mfinanciacion_limite) , na.rm=TRUE ) ]
  dataset[ , mv_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , mv_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]
  dataset[ , mv_msaldototal          := rowSums( cbind( Master_msaldototal,  Visa_msaldototal) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumospesos       := rowSums( cbind( Master_mconsumospesos,  Visa_mconsumospesos) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumosdolares     := rowSums( cbind( Master_mconsumosdolares,  Visa_mconsumosdolares) , na.rm=TRUE ) ]
  dataset[ , mv_mlimitecompra        := rowSums( cbind( Master_mlimitecompra,  Visa_mlimitecompra) , na.rm=TRUE ) ]
  dataset[ , mv_mpagado              := rowSums( cbind( Master_mpagado,  Visa_mpagado) , na.rm=TRUE ) ]
  dataset[ , mv_cconsumos            := rowSums( cbind( Master_cconsumos,  Visa_cconsumos) , na.rm=TRUE ) ]
  dataset[ , mv_mpagominimo          := rowSums( cbind( Master_mpagominimo,  Visa_mpagominimo) , na.rm=TRUE ) ]

  #combino los margenes y los dividos por la antiguedad para obtener los margenes de ganancia promedio por mes
  dataset[ , fe_ganancia_margen_banco := rowSums( cbind( mactivos_margen,  mpasivos_margen) , na.rm=TRUE ) ]
  dataset[ , fe_avg_mes_ganancia_margen_banco := fe_ganancia_margen_banco /  cliente_antiguedad ]
    
  #combino los montos de todas las cuentas del cliente
  dataset[ , fe_montos_cuentas := rowSums( cbind(mcuenta_corriente_adicional, mcuenta_corriente, mcaja_ahorro, mcaja_ahorro_adicional, mcaja_ahorro_dolares) , na.rm=TRUE ) ]
    
  #combino la cantidad de prestamos
  dataset[ , fe_cantidad_prestamos := rowSums( cbind( cprestamos_personales, cprestamos_prendarios, cprestamos_hipotecarios) , na.rm=TRUE ) ]
    
  #combino el total de deuda
  dataset[ , fe_total_deuda := rowSums( cbind( mprestamos_personales, mprestamos_prendarios, mprestamos_hipotecarios) , na.rm=TRUE ) ]  
   
  #combino los montos de plazo fijo vigentes
  dataset[ , fe_montos_plazos_fijos := rowSums( cbind( mplazo_fijo_dolares, mplazo_fijo_pesos) , na.rm=TRUE ) ]
   
  #combino los montos de inversiones
  dataset[ , fe_montos_inversion := rowSums( cbind( minversion1_pesos, minversion1_dolares, minversion2) , na.rm=TRUE ) ]
    
  #combino la cantidad de seguros
  dataset[ , fe_cantidad_seguros := rowSums( cbind( cseguro_vida, cseguro_auto, cseguro_vivienda, cseguro_accidentes_personales) , na.rm=TRUE ) ]
   
  #combino las acreditaciones del cliente
  dataset[ , fe_acreditaciones := rowSums( cbind( mpayroll, mpayroll2) , na.rm=TRUE ) ]   
    
    
  columnas_extendidas <-  copy( setdiff(  colnames(dataset), columnas_originales ) )

  #grabo con nombre extendido
  fwrite( dataset,
          file=arch_destino,
          sep= "," )
}
#------------------------------------------------------------------------------

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <-  "./datasets/"    #"./buckets/b1/datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")
karchivo_salida      <-  paste0(kcarpeta_datasets, "competencia1_2022_FE_test.csv")

#lectura rapida del dataset  usando fread  de la libreria  data.table
dataset1  <- fread(karchivo_entrada)

EnriquecerDataset( dataset1, karchivo_salida )

#Excluyo los campos con los cuales genere los FE
dataset_filter  <- fread  ("./datasets/competencia1_2022_FE_test.csv")

dataset2  <-  dataset_filter[,.SD, .SDcols = !c("mactivos_margen", "mpasivos_margen", "mcuenta_corriente_adicional", "mcuenta_corriente", "mcaja_ahorro", "mcaja_ahorro_adicional", "mcaja_ahorro_dolares", "cprestamos_personales", "mprestamos_personales", "cprestamos_prendarios", "mprestamos_prendarios", "cprestamos_hipotecarios", "mprestamos_hipotecarios", "mplazo_fijo_dolares", "mplazo_fijo_pesos", "minversion1_pesos", "minversion1_dolares", "minversion2", "cseguro_vida", "cseguro_auto", "cseguro_vivienda", "cseguro_accidentes_personales", "mpayroll", "mpayroll2", "Master_delinquency", "Master_status", "Master_mfinanciacion_limite", "Master_Fvencimiento", "Master_Finiciomora", "Master_msaldototal", "Master_msaldopesos", "Master_msaldodolares", "Master_mconsumospesos", "Master_mconsumosdolares", "Master_mlimitecompra", "Master_madelantopesos", "Master_madelantodolares", "Master_fultimo_cierre", "Master_mpagado", "Master_mpagospesos", "Master_mpagosdolares", "Master_fechaalta", "Master_mconsumototal", "Master_cconsumos", "Master_cadelantosefectivo", "Master_mpagominimo", "Visa_delinquency", "Visa_status", "Visa_mfinanciacion_limite", "Visa_Fvencimiento", "Visa_Finiciomora", "Visa_msaldototal", "Visa_msaldopesos", "Visa_msaldodolares", "Visa_mconsumospesos", "Visa_mconsumosdolares", "Visa_mlimitecompra", "Visa_madelantopesos", "Visa_madelantodolares", "Visa_fultimo_cierre", "Visa_mpagado", "Visa_mpagospesos", "Visa_mpagosdolares", "Visa_fechaalta", "Visa_mconsumototal", "Visa_cconsumos", "Visa_cadelantosefectivo", "Visa_mpagominimo")]

fwrite( dataset2,
        file= "./datasets/competencia1_2022_FE_test1.csv",
        sep= ",")


quit( save="no")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,665611,35.6,1286368,68.7,1286368,68.7
Vcells,1251411,9.6,61243418,467.3,76276626,582.0
